# List of Exposures with Hologram at USDF


- work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2023/07/18



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_27/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/ctrl_mpexec/g88183d3282+88d805ed8c/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/spectractor/g7bf34a7a6e/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/obs_lsst/gfd7e96728c+6f6aa72e7b/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/pipe_tasks/gfb267207ab+bb26f3254a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/shapelet/gd4112516d0+5ce2cc5508/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/meas_modelfit/g807e3ba568+b7d80ac9cf/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/proxmin/g33b4157f25/l

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_27/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/eups 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2023_27
   gdf42428520+4e9f6d16c8 	current w_2023_27 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','science_program','jd','mjd'])

In [12]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    #df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,info.timespan.begin.to_string(),info.timespan.end.to_string() ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
    #                         info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program]
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [13]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023071900120,
 'physical_filter': 'SDSSg_65mm~empty',
 'obs_id': 'AT_O_20230719_000120',
 'exposure_time': 6.0,
 'dark_time': 6.22911,
 'observation_type': 'flat',
 'observation_reason': 'flat',
 'day_obs': 20230719,
 'seq_num': 120,
 'seq_start': 120,
 'seq_end': 120,
 'group_name': '2023-07-19T22:16:48.597',
 'group_id': 2697346085970000,
 'target_name': 'FlatField position',
 'science_program': 'unknown',
 'tracking_ra': 191.17044695153936,
 'tracking_dec': -80.98394519573743,
 'sky_angle': None,
 'azimuth': 181.8056477580152,
 'zenith_angle': 50.98221431006178,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-07-19 22:16:48.678000', scale='tai', format='iso'), end=astropy.time.Time('2023-07-19 22:16:54.907000', scale='tai', format='iso'))}

In [14]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51482,2023071900116,AT_O_20230719_000116,20230719,116,2023-07-19 22:16:13.785008,2023-07-19 22:16:20.024,flat,FlatField position,SDSSg_65mm~empty,50.981877,6.0,191.026926,-80.983563,NaN,unknown,2.460145e+06,60144.927937
51483,2023071900117,AT_O_20230719_000117,20230719,117,2023-07-19 22:16:22.512018,2023-07-19 22:16:28.756,flat,FlatField position,SDSSg_65mm~empty,50.981963,6.0,191.063862,-80.983661,NaN,unknown,2.460145e+06,60144.928038
51484,2023071900118,AT_O_20230719_000118,20230719,118,2023-07-19 22:16:31.237982,2023-07-19 22:16:37.467,flat,FlatField position,SDSSg_65mm~empty,50.982050,6.0,191.100740,-80.983758,NaN,unknown,2.460145e+06,60144.928139
51485,2023071900119,AT_O_20230719_000119,20230719,119,2023-07-19 22:16:39.956019,2023-07-19 22:16:46.190,flat,FlatField position,SDSSg_65mm~empty,50.982138,6.0,191.137639,-80.983858,NaN,unknown,2.460145e+06,60144.928240


In [15]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [16]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51482,2023071900116,AT_O_20230719_000116,20230719,116,2023-07-19 22:16:13.785008,2023-07-19 22:16:20.024,flat,FlatField position,SDSSg_65mm~empty,50.981877,6.0,191.026926,-80.983563,NaN,unknown,2.460145e+06,60144.927937
51483,2023071900117,AT_O_20230719_000117,20230719,117,2023-07-19 22:16:22.512018,2023-07-19 22:16:28.756,flat,FlatField position,SDSSg_65mm~empty,50.981963,6.0,191.063862,-80.983661,NaN,unknown,2.460145e+06,60144.928038
51484,2023071900118,AT_O_20230719_000118,20230719,118,2023-07-19 22:16:31.237982,2023-07-19 22:16:37.467,flat,FlatField position,SDSSg_65mm~empty,50.982050,6.0,191.100740,-80.983758,NaN,unknown,2.460145e+06,60144.928139
51485,2023071900119,AT_O_20230719_000119,20230719,119,2023-07-19 22:16:39.956019,2023-07-19 22:16:46.190,flat,FlatField position,SDSSg_65mm~empty,50.982138,6.0,191.137639,-80.983858,NaN,unknown,2.460145e+06,60144.928240


## Selection of science

In [17]:
df_science = df_exposure[df_exposure.type == 'science']

In [18]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_2827/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [19]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
51250,2023071800267,AT_O_20230718_000267,20230718,267,2023-07-19 00:38:55.292986,2023-07-19 00:39:25.532,science,Photo1600-1_007,SDSSi_65mm~empty,7.574884,30.0,240.039389,-24.048276,0.000102,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.027029
51251,2023071800268,AT_O_20230718_000268,20230718,268,2023-07-19 00:39:39.637988,2023-07-19 00:40:09.875,science,Photo1600-1_007,SDSSg_65mm~empty,7.480142,30.0,240.039460,-24.048186,0.000159,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.027542
51252,2023071800269,AT_O_20230718_000269,20230718,269,2023-07-19 00:40:23.948993,2023-07-19 00:40:54.188,science,Photo1600-1_007,SDSSr_65mm~empty,7.387707,30.0,240.039503,-24.048180,0.000610,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.028055
51253,2023071800270,AT_O_20230718_000270,20230718,270,2023-07-19 00:41:09.987005,2023-07-19 00:41:40.231,science,Photo1600-1_027,SDSSr_65mm~empty,7.338247,30.0,239.974757,-23.959410,0.000288,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.028588
51254,2023071800271,AT_O_20230718_000271,20230718,271,2023-07-19 00:41:54.382982,2023-07-19 00:42:24.623,science,Photo1600-1_027,SDSSg_65mm~empty,7.252577,30.0,239.974662,-23.959544,0.000539,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.029102
51255,2023071800272,AT_O_20230718_000272,20230718,272,2023-07-19 00:42:38.749992,2023-07-19 00:43:08.995,science,Photo1600-1_027,SDSSi_65mm~empty,7.168016,30.0,239.974690,-23.959447,0.000473,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.029615
51256,2023071800273,AT_O_20230718_000273,20230718,273,2023-07-19 00:45:58.934020,2023-07-19 00:46:29.163,science,Photo1800-1_011,SDSSi_65mm~empty,29.924899,30.0,270.174550,-25.018621,0.001279,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.031932
51257,2023071800274,AT_O_20230718_000274,20230718,274,2023-07-19 00:46:43.287993,2023-07-19 00:47:13.532,science,Photo1800-1_011,SDSSg_65mm~empty,29.769424,30.0,270.174506,-25.018607,0.001085,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.032445
51258,2023071800275,AT_O_20230718_000275,20230718,275,2023-07-19 00:47:27.650014,2023-07-19 00:47:57.883,science,Photo1800-1_011,SDSSr_65mm~empty,29.603417,30.0,270.174491,-25.018603,0.001164,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.032959
51259,2023071800276,AT_O_20230718_000276,20230718,276,2023-07-19 00:50:41.990018,2023-07-19 00:51:12.233,science,Photo1600-1_029,SDSSr_65mm~empty,6.472191,30.0,240.007067,-23.959509,359.999703,AUXTEL_PHOTO_IMAGING,2.460145e+06,60144.035208


In [20]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706, 20230718])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [21]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
#date_sel= 20230131

# data in February
#date_sel = 20230201
#date_sel = 20230202
#date_sel = 20230704

#date_sel = 20230510
#date_sel = 20230509
#date_sel = 20230511

#date_sel = 20230705
#date_sel = 20230706

date_sel = 20230718


In [22]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [23]:
len(df_science_selected)

103

# List of filters

In [24]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['SDSSr_65mm~empty', 'SDSSg_65mm~empty', 'SDSSi_65mm~empty',
       'empty~holo4_003'], dtype=object)

In [25]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [26]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [27]:
pd.set_option('display.max_rows', None)

In [28]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20230718_filt_empty-holo4_003.list


/tmp/ipykernel_2827/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [29]:
N = len(all_def_science_selected_final)

In [30]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
51350,2023071800368,AT_O_20230718_000368,20230718,368,2023-07-19 03:09:48.085002,2023-07-19 03:10:18.321,science,HD 192814,empty~holo4_003,28.116268,30.0,304.435476,-25.969373,1.358934,spec,2.460145e+06,60144.131807
51351,2023071800369,AT_O_20230718_000369,20230718,369,2023-07-19 03:10:31.093016,2023-07-19 03:11:01.330,science,HD 192814,empty~holo4_003,27.961173,30.0,304.435517,-25.969369,1.359021,spec,2.460145e+06,60144.132304
51355,2023071800373,AT_O_20230718_000373,20230718,373,2023-07-19 03:16:05.905000,2023-07-19 03:16:36.141,science,HD146233,empty~holo4_003,35.921710,30.0,243.875745,-8.403517,216.860355,spec,2.460145e+06,60144.136179
51356,2023071800374,AT_O_20230718_000374,20230718,374,2023-07-19 03:16:48.816013,2023-07-19 03:17:19.062,science,HD146233,empty~holo4_003,36.054903,30.0,243.875749,-8.403526,216.860338,spec,2.460145e+06,60144.136676
51360,2023071800378,AT_O_20230718_000378,20230718,378,2023-07-19 03:41:18.064008,2023-07-19 03:41:48.308,science,HD165763,empty~holo4_003,11.870981,30.0,272.080466,-21.278435,225.885820,BLOCK-59,2.460145e+06,60144.153681
51361,2023071800379,AT_O_20230718_000379,20230718,379,2023-07-19 03:42:01.044986,2023-07-19 03:42:31.275,science,HD165763,empty~holo4_003,11.982162,30.0,272.080479,-21.278457,225.885877,BLOCK-59,2.460145e+06,60144.154179


In [31]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

In [32]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

In [33]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [34]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [35]:
df_forsave = pd.concat(all_def_science_selected_final)

In [36]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [37]:
filename_out_csv

'holosummary_20230718.csv'

In [38]:
df_forsave.to_csv(filename_out_csv) 

In [39]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [40]:
df_forsave.to_excel(filename_out_excel) 